In [2]:
import pandas as pd
from sklearn.metrics import jaccard_score, pairwise_distances
import numpy as np

In [3]:
users=["User1", "User2", "User3","User4", "User5"]
items=["Item A", "Item B", "Item C", "Item D", "Item E"]
data=[
    [1, 0, 1, 1, 0],
    [1, 0, 0, 1, 1],
    [1, 0, 1, 0, 0],
    [0, 1, 0, 1, 1],
    [1, 1, 1, 0, 1]
]
df = pd.DataFrame(data, columns = items, index = users)
df

,Item A,Item B,Item C,Item D,Item E
User1,1,0,1,1,0
User2,1,0,0,1,1
User3,1,0,1,0,0
User4,0,1,0,1,1
User5,1,1,1,0,1


In [4]:
df.T

,User1,User2,User3,User4,User5
Item A,1,1,1,0,1
Item B,0,0,0,1,1
Item C,1,0,1,0,1
Item D,1,1,0,1,0
Item E,0,1,0,1,1


##### 直接计算两项的系数

In [5]:
jaccard_score(df['Item A'], df['Item B'])

0.2

In [6]:
jaccard_score(df['Item B'], df['Item C'])

0.25

##### 所有数据两两的系数

In [7]:
user_similarity = 1 - pairwise_distances(df.values, metric='jaccard')
user_similarity = pd.DataFrame(user_similarity, columns=users, index = users)
user_similarity

/Users/zhuoyujiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


,User1,User2,User3,User4,User5
User1,1.000000,0.50,0.666667,0.2,0.4
User2,0.500000,1.00,0.250000,0.5,0.4
User3,0.666667,0.25,1.000000,0.0,0.5
User4,0.200000,0.50,0.000000,1.0,0.4
User5,0.400000,0.40,0.500000,0.4,1.0


###### 计算物品相似度

In [8]:
item_similarity = 1- pairwise_distances(df.values.T, metric = 'jaccard')
item_similarity = pd.DataFrame(item_similarity, columns=items, index = items)
item_similarity

/Users/zhuoyujiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


,Item A,Item B,Item C,Item D,Item E
Item A,1.00,0.200000,0.75,0.40,0.400000
Item B,0.20,1.000000,0.25,0.25,0.666667
Item C,0.75,0.250000,1.00,0.20,0.200000
Item D,0.40,0.250000,0.20,1.00,0.500000
Item E,0.40,0.666667,0.20,0.50,1.000000


In [9]:
topN_user={}
for i in user_similarity.index:
    # 取出每一列数据，并删除自身，然后排序
    column_after_sort=sorted(user_similarity.loc[i].drop([i]), reverse=True)[:2]
    topN_user[i] =column_after_sort
topN_user

{'User1': [0.6666666666666667, 0.5],
 'User2': [0.5, 0.5],
 'User3': [0.6666666666666667, 0.5],
 'User4': [0.5, 0.4],
 'User5': [0.5, 0.4]}

In [10]:
# aim= [0.5, 0.4]
# have = user_similarity.loc[:, 'User1'].to_list()
# index_num=[have.index(x) for x in aim]
#
# index = [user_similarity.index[i] for i in index_num]
# #df.T['User1'].replace(0, np.nan).dropna().index
# index+['User1']

In [11]:
# #list(df.T['User1'].replace(0, np.nan).dropna().index)
# similar_user =[]
# for x, y in user_similarity.items:
#     for each in y:
#         if user_similarity.loc[:, x] == each:
#             similar_user.append()
#         print(x)

In [12]:
# for values in topN_user:
#    # print(key)
#     print(values)

In [13]:
# result_dict={}
# for x in topN_user:
#    # print(x)
#    # similarity_num = user_similarity.loc[:, x].to_list()
#     user_similarity.loc[user_similarity[x] ==topN_user[x],]
#    # for y in similarity_num:
#    #     if y not in topN_user[x]:
#    #         result = user_similarity[x].replace(y, np.nan).dropna()
#     #    result_index = result.index
#     print(user_similarity)
#    # result_dict[x] = result_index
# #result_dict

In [14]:
# 构建推荐用户结果
recommend_result = {}
for x, y in topN_user.items():
    index_result = user_similarity.loc[user_similarity[x].isin(y)].index.tolist()
    recommend_result[x] = index_result
recommend_result

{'User1': ['User2', 'User3'],
 'User2': ['User1', 'User4'],
 'User3': ['User1', 'User5'],
 'User4': ['User2', 'User5'],
 'User5': ['User1', 'User2', 'User3', 'User4']}

In [47]:
# 协同推荐用户
user_filtered={}
user_set=set()
for user, recommend_users in recommend_result.items():
    for each in recommend_users:
        user_set = user_set.union(set(recommend_result[each]))
        user_set -= set(recommend_users)
        user_filtered[user] = list(user_set-set(user))
    user_filtered[user].remove(user)
user_filtered

{'User1': ['User4', 'User5'],
 'User2': ['User3', 'User5'],
 'User3': ['User4', 'User2'],
 'User4': ['User3', 'User1'],
 'User5': []}

In [53]:
# 过滤掉已有的推荐
item_filtered={}
item_set = set()
for key, value in user_filtered.items():
    remove_set = set(df.T[key].replace(0, np.nan).dropna().index)
    for u in value:
        item_set = item_set.union(set(df.T[u].replace(0, np.nan).dropna().index))
        item_set -= remove_set
        item_filtered[key] = list(item_set)
item_filtered

{'User1': ['Item E', 'Item B'],
 'User2': ['Item C', 'Item B'],
 'User3': ['Item E', 'Item D', 'Item B'],
 'User4': ['Item A', 'Item C']}

##### continuous data eg.rating

In [57]:
data2=[
    [5,3,4,4,None],
    [3,1,2,3,3],
    [4,3,4,3,5],
    [3,3,1,5,4],
    [1,5,5,2,1]
]
df2=pd.DataFrame(data2, columns=items, index=users)
user_similar = round(df2.T.corr(),4)
user_similar
# 用户2对某一电影的评分可以根据和他最相似的（用户1评分*相似度+用户4评分*相似度）/相似度之和

,User1,User2,User3,User4,User5
User1,1.0000,0.8528,0.7071,0.0000,-0.7921
User2,0.8528,1.0000,0.4677,0.4900,-0.9001
User3,0.7071,0.4677,1.0000,-0.1612,-0.4666
User4,0.0000,0.4900,-0.1612,1.0000,-0.6415
User5,-0.7921,-0.9001,-0.4666,-0.6415,1.0000


In [56]:
item_similar = round(df2.corr(),4)

##### ALS矩阵分解